In [1]:
from tqdm import tqdm
import matplotlib.pyplot as plt # type: module
import matplotlib.ticker as ticker

import numpy as np
import os, glob
import time
import warnings

from rur.fortranfile import FortranFile
from rur import uri, uhmi, painter, drawer
from rur.sci.photometry import measure_luminosity
from rur.sci.geometry import get_angles, euler_angle
from rur.utool import rotate_data
from scipy.ndimage import gaussian_filter
uri.timer.verbose=1
# from rur.sci.kinematics import f_getpot

from icl_IO import mode2repo, pklsave, pklload
from icl_tool import *
from icl_numba import large_isin, large_isind, isin
from icl_draw import drawsnap, add_scalebar, addtext, MakeSub_nolabel, label_to_in, fancy_axis, circle
import argparse, subprocess
from importlib import reload
import cmasher as cmr
from copy import deepcopy

In [3]:
mode = 'nh'
iout = 1026
repo, rurmode, dp = mode2repo(mode)
snap = uri.RamsesSnapshot(repo, iout, mode=rurmode)
snaps = uri.TimeSeries(snap)
snaps.read_iout_avail()
nout = snaps.iout_avail['iout']
gals = uhmi.HaloMaker.load(snap, galaxy=True, double_precision=dp)
hals = uhmi.HaloMaker.load(snap, galaxy=False, double_precision=dp)
database = f"/home/jeon/MissingSat/database"

[Output 01026] Age (Gyr) : 11.624 / 13.741, z = 0.17149 (a = 0.8536)


In [52]:
LG = pklload(f"{database}/00_LocalGroup_final.pickle")


In [24]:
for key in LG.keys():
    BGG = LG[key]['BGG']
    sats = LG[key]['sats']
    if(0 in sats['id']):
        print(f"[{key:04d}]")
        sats = sats[sats['id'] != 0]
        LG[key]['sats'] = sats
pklsave(LG, f"{database}/00_LocalGroup_final.pickle", overwrite=True)

# HaloID = Host?

In [70]:
for key in LG.keys():
    print(f"[{key:04d}]")
    BGG = LG[key]['BGG']
    sats = LG[key]['sats']
    uniq, counts = np.unique(sats['id'], return_counts=True)
    subs = LG[key]['subs']
    real = LG[key]['real']
    LG[key]['parking'] = []

    # fig, ax = plt.subplots()
    # ax.scatter(sats['x'], sats['y'], color='b', ec='none', s=4, alpha=0.5)
    # ax.scatter(subs['x'], subs['y'], color='k', ec='none', s=4, alpha=0.5)
    # show=False
    for sat in sats:
        where = np.where(real['gid'] == sat['id'])[0][0]
        if(not sat['halo_id'] == real[where]['hid']):
            if(not sat['halo_id'] in subs['id'])and(sat['halo_id'] != BGG['halo_id']):
                host = hals[hals['id'] == sat['halo_id']][0]
                dist = distance(host, sat)
                if(dist < host['rvir']-sat['r']):
                    ingals = cut_sphere(gals, host['x'], host['y'], host['z'], host['rvir'])
                    ingals = ingals[ingals['id'] != sat['id']]
                    if(len(ingals)==0):
                        print( f"({sat['id']}) {sat['halo_id']} != {real[where]['hid']}" )
                        LG[key]['parking'].append((sat['id'], sat['halo_id']))
                        
    #                     cir = plt.Circle((host['x'], host['y']), host['rvir'], color='k', fill=False)
    #                     ax.add_patch(cir)
    #                     cir = plt.Circle((sat['x'], sat['y']), sat['r'], color='r', fill=False)
    #                     ax.add_patch(cir)
    #                     show=True
    # if(show): plt.show()
    # plt.close()

    #         stop()

    # stop()

[0001]
(606) 20816 != 0
[0002]
(726) 22271 != 0
(757) 22013 != 0
(831) 22320 != 0
[0003]
[0004]
(520) 20135 != 0
(566) 19810 != 0
(1065) 20172 != 0
[0007]
[0008]
[0009]
(925) 23375 != 0
[0010]
[0014]
(689) 26899 != 0
[0017]
[0023]
[0024]
[0025]
[0026]
[0029]
[0035]
[0036]
[0037]
[0040]
[0046]
[0047]
[0052]
[0063]
[0064]
(643) 26084 != 0
[0075]
[0084]
[0092]
[0107]
[0108]
[0121]
[0129]
[0207]
[0011]


In [78]:
def calc_rhalf_sorted(sorted_dist, sorted_weights, ratio=0.5):
    sw = np.cumsum(sorted_weights)
    sw /= sw[-1]
    return sorted_dist[np.argmin(np.abs(sw-ratio))]

In [81]:
dtype2 = [
    ("r10_mem", np.float64), ("r50_mem", np.float64), ("r90_mem", np.float64),
    ("r10_vir", np.float64), ("r50_vir", np.float64), ("r90_vir", np.float64),
    ("r10_max", np.float64), ("r50_max", np.float64), ("r90_max", np.float64),
    ]
for key in LG.keys():
    parking = LG[key]['parking']
    if(len(parking)==0): continue
    print(f"[{key:04d}]")
    BGG = LG[key]['BGG']
    sats = LG[key]['sats']
    uniq, counts = np.unique(sats['id'], return_counts=True)
    subs = LG[key]['subs']
    real = LG[key]['real']
    
    star=None; dm=None; cell=None

    dtype = subs.dtype
    newsubs = np.zeros(len(parking), dtype=dtype)
    for i,ipark in enumerate(parking):
        gid, hid = ipark
        print(i,gid,hid)
        where = np.where(real['gid'] == gid)[0][0]
        real[where]['hid'] = hid# <---------------
        host = hals[hid-1]
        for iname in host.dtype.names:
            newsubs[i][iname] = host[iname]
        if(os.path.exists(f"{database}/parts/insub/nh_dm_{key:04d}_{hid:07d}.pickle")):
            alldm = pklload(f"{database}/parts/insub/nh_dm_{key:04d}_{hid:07d}.pickle")
            alldm = uri.Particle(alldm,snap)
        else:
            if(star is None): star = uri.Particle(pklload(f"./database/parts/nh_star_{key:04d}.pickle"), snap)
            if(cell is None): cell = uri.Particle(pklload(f"./database/parts/nh_cell_{key:04d}.pickle"), snap)
            if(dm is None): dm = uri.Particle(pklload(f"./database/parts/nh_dm_{key:04d}.pickle"), snap)
            istar = cut_sphere(star, host['x'],host['y'],host['z'],host['r'])
            icell = cut_sphere(cell, host['x'],host['y'],host['z'],host['r'])
            idm = cut_sphere(dm, host['x'],host['y'],host['z'],host['r'])
            pklsave(istar.table, f"./database/parts/insub/nh_star_{key:04d}_{host['id']:07d}.pickle")
            pklsave(icell.table, f"./database/parts/insub/nh_cell_{key:04d}_{host['id']:07d}.pickle")
            pklsave(idm.table, f"./database/parts/insub/nh_dm_{key:04d}_{host['id']:07d}.pickle")
            alldm = pklload(f"{database}/parts/insub/nh_dm_{key:04d}_{hid:07d}.pickle")
            alldm = uri.Particle(alldm,snap)

        alldist = distance(host, alldm); argsort = np.argsort(alldist)
        alldist = alldist[argsort]; allmass = alldm['m'][argsort]
        memdm = uhmi.HaloMaker.read_member_part(snap, host['id'], galaxy=False, target_fields=['x','y','z','m'])
        memdist = distance(host, memdm); argsort = np.argsort(memdist)
        memdist = memdist[argsort]; memmass = memdm['m'][argsort]

        newsubs[i]['r10_mem'] = calc_rhalf_sorted(memdist, memmass, ratio=0.1)
        newsubs[i]['r50_mem'] = calc_rhalf_sorted(memdist, memmass, ratio=0.5)
        newsubs[i]['r90_mem'] = calc_rhalf_sorted(memdist, memmass, ratio=0.9)
        newsubs[i]['r10_max'] = calc_rhalf_sorted(alldist, allmass, ratio=0.1)
        newsubs[i]['r50_max'] = calc_rhalf_sorted(alldist, allmass, ratio=0.5)
        newsubs[i]['r90_max'] = calc_rhalf_sorted(alldist, allmass, ratio=0.9)
        virdm, ind = cut_sphere(alldm, host['x'], host['y'], host['z'], host['rvir'], return_index=True)
        alldist = alldist[ind]; allmass = allmass[ind]
        newsubs[i]['r10_vir'] = calc_rhalf_sorted(alldist, allmass, ratio=0.1)
        newsubs[i]['r50_vir'] = calc_rhalf_sorted(alldist, allmass, ratio=0.5)
        newsubs[i]['r90_vir'] = calc_rhalf_sorted(alldist, allmass, ratio=0.9)
    LG[key]['subs'] = np.hstack((subs, newsubs))
pklsave(LG, f"{database}/00_LocalGroup_final.pickle", overwrite=True)

[0001]
0 606 20816
[0002]
0 726 22271
1 757 22013
2 831 22320
[0004]
0 520 20135
1 566 19810
2 1065 20172
[0009]
0 925 23375
[0014]
0 689 26899
[0064]
0 643 26084


In [89]:
MASSs = pklload(f"{database}/01_mass_in_sub.pickle")
MASSs_rvir = pklload(f"{database}/01_mass_in_sub_rvir.pickle")

In [90]:
for key in LG.keys():
    BGG = LG[key]['BGG']
    sats = LG[key]['sats']
    uniq, counts = np.unique(sats['id'], return_counts=True)
    subs = LG[key]['subs']
    real = LG[key]['real']
    
    iMASS = MASSs[MASSs['host'] == key]
    iMASS_rvir = MASSs_rvir[MASSs_rvir['host'] == key]

    subids = []
    for sub in subs:
        if(not sub['id'] in iMASS['sub']):
            subids.append(sub['id'])
    
    count = len(subids)
    if(count>0):
        print(f"[{key:04d}]")
        newMass = np.zeros(count, dtype=MASSs.dtype)
        newMass['host'] = key
        newMass_rvir = np.zeros(count, dtype=MASSs.dtype)
        newMass_rvir['host'] = key
        for i, subid in enumerate(subids):
            newMass[i]['sub'] = subid
            
            print(key, subid)
            istar = uri.Particle(pklload(f"./database/parts/insub/nh_star_{key:04d}_{subid:07d}.pickle"), snap)
            if(len(istar)>0): newMass[i]['mstar'] = np.sum(istar['m','Msol'])
            istar = cut_sphere(istar, sub['x'],sub['y'],sub['z'],sub['rvir'])
            if(len(istar)>0): newMass_rvir[i]['mstar'] = np.sum(istar['m','Msol'])

            idm = uri.Particle(pklload(f"./database/parts/insub/nh_dm_{key:04d}_{subid:07d}.pickle"), snap)
            if(len(idm)>0): newMass[i]['mdm'] = np.sum(idm['m','Msol'])
            idm = cut_sphere(idm, sub['x'],sub['y'],sub['z'],sub['rvir'])
            if(len(idm)>0): newMass_rvir[i]['mdm'] = np.sum(idm['m','Msol'])

            icell = uri.Cell(pklload(f"./database/parts/insub/nh_cell_{key:04d}_{subid:07d}.pickle"), snap)
            if(len(icell)>0): newMass[i]['mcell'] = np.sum(icell['m','Msol'])
            icold = icell[icell['T','K'] < 2e4]
            if(len(icold)>0): newMass[i]['mcold'] = np.sum(icold['m','Msol'])
            icell = cut_sphere(icell, sub['x'],sub['y'],sub['z'],sub['rvir'])
            if(len(icell)>0): newMass_rvir[i]['mcell'] = np.sum(icell['m','Msol'])
            icold = icell[icell['T','K'] < 2e4]
            if(len(icold)>0): newMass_rvir[i]['mcold'] = np.sum(icold['m','Msol'])
        MASSs = np.hstack((MASSs, newMass))
        MASSs_rvir = np.hstack((MASSs_rvir, newMass_rvir))
pklsave(MASSs, f"{database}/01_mass_in_sub.pickle", overwrite=True)
pklsave(MASSs_rvir, f"{database}/01_mass_in_sub_rvir.pickle", overwrite=True)

[0001]
1 20816
[0002]
2 22271
2 22013
2 22320
[0004]
4 20135
4 19810
4 20172
[0009]
9 23375
[0014]
14 26899
[0064]
64 26084


In [91]:
MASSs.dtype

dtype([('host', '<i4'), ('sub', '<i8'), ('dink', '?'), ('mdm', '<f8'), ('mstar', '<f8'), ('mcold', '<f8'), ('mcell', '<f8')])

In [92]:
subs.dtype

dtype([('nparts', '<i4'), ('id', '<i4'), ('timestep', '<i4'), ('level', '<i4'), ('host', '<i4'), ('hostsub', '<i4'), ('nbsub', '<i4'), ('nextsub', '<i4'), ('aexp', '<f8'), ('m', '<f8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('vx', '<f8'), ('vy', '<f8'), ('vz', '<f8'), ('Lx', '<f8'), ('Ly', '<f8'), ('Lz', '<f8'), ('r', '<f8'), ('a', '<f8'), ('b', '<f8'), ('c', '<f8'), ('ek', '<f8'), ('ep', '<f8'), ('et', '<f8'), ('spin', '<f8'), ('sigma', '<f8'), ('rvir', '<f8'), ('mvir', '<f8'), ('tvir', '<f8'), ('cvel', '<f8'), ('rho0', '<f8'), ('rc', '<f8'), ('mcontam', '<f8'), ('r10_mem', '<f8'), ('r50_mem', '<f8'), ('r90_mem', '<f8'), ('r10_vir', '<f8'), ('r50_vir', '<f8'), ('r90_vir', '<f8'), ('r10_max', '<f8'), ('r50_max', '<f8'), ('r90_max', '<f8')])

In [96]:
for key in LG.keys():
    BGG = LG[key]['BGG']
    sats = LG[key]['sats']
    uniq, counts = np.unique(sats['id'], return_counts=True)
    subs = LG[key]['subs']
    real = LG[key]['real']

    dtype1 = subs.dtype
    dtype2 = MASSs.dtype.descr[1:]+[('Host', '<i4')]
    dtype = np.dtype(dtype1.descr + dtype2)
    newsubs = np.zeros(len(subs), dtype=dtype)
    for iname in subs.dtype.names:
        newsubs[iname] = subs[iname]
    iMass = MASSs[MASSs['host'] == key]
    for isub in newsubs:
        where = np.where(iMass['sub'] == isub['id'])[0][0]
        isub['Host'] = iMass[where]['host']
        isub['dink'] = iMass[where]['dink']
        isub['mstar'] = iMass[where]['mstar']
        isub['mdm'] = iMass[where]['mdm']
        isub['mcell'] = iMass[where]['mcell']
        isub['mcold'] = iMass[where]['mcold']
    LG[key]['subs'] = newsubs


In [97]:
pklsave(LG, f"{database}/00_LocalGroup_final.pickle", overwrite=True)

In [100]:
print( LG[1]['BGG'].dtype.names )

('nparts', 'id', 'timestep', 'level', 'host', 'hostsub', 'nbsub', 'nextsub', 'aexp', 'm', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'Lx', 'Ly', 'Lz', 'r', 'a', 'b', 'c', 'ek', 'ep', 'et', 'spin', 'sigma', 'sigma_bulge', 'm_bulge', 'rvir', 'mvir', 'tvir', 'cvel', 'rho0', 'rc', 'halo_id', 'halo_nparts', 'halo_level', 'halo_host', 'halo_hostsub', 'halo_x', 'halo_y', 'halo_z', 'halo_vx', 'halo_vy', 'halo_vz', 'halo_mvir', 'halo_rvir', 'fcontam', 'dist', 'central', 'main', 'r200', 'm200', 'r200_code', 'm_star_200', 'm_gas_200', 'fcontam_200', 'rp', 'sfr', 'sfr_tot', 'galaxy_nh2', 'halo_nh2', 'matchrate')


In [99]:
print( LG[1]['sats'].dtype.names )

('nparts', 'id', 'timestep', 'level', 'host', 'hostsub', 'nbsub', 'nextsub', 'aexp', 'm', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'Lx', 'Ly', 'Lz', 'r', 'a', 'b', 'c', 'ek', 'ep', 'et', 'spin', 'sigma', 'sigma_bulge', 'm_bulge', 'rvir', 'mvir', 'tvir', 'cvel', 'rho0', 'rc', 'halo_id', 'halo_nparts', 'halo_level', 'halo_host', 'halo_hostsub', 'halo_x', 'halo_y', 'halo_z', 'halo_vx', 'halo_vy', 'halo_vz', 'halo_mvir', 'halo_rvir', 'fcontam', 'dist', 'central', 'main', 'Host', 'r50m', 'r90m', 'r50r', 'r90r', 'SFR_mem', 'u_mem', 'g_mem', 'r_mem', 'i_mem', 'z_mem', 'metal_mem', 'ager_mem', 't50_mem', 't90_mem', 'SFR_r50m', 'u_r50m', 'g_r50m', 'r_r50m', 'i_r50m', 'z_r50m', 'metal_r50m', 'ager_r50m', 't50_r50m', 't90_r50m', 'mgas_r50m', 'mcold_r50m', 'mdm_r50m', 'SFR_r90m', 'u_r90m', 'g_r90m', 'r_r90m', 'i_r90m', 'z_r90m', 'metal_r90m', 'ager_r90m', 't50_r90m', 't90_r90m', 'mgas_r90m', 'mcold_r90m', 'mdm_r90m', 'SFR_r50r', 'u_r50r', 'g_r50r', 'r_r50r', 'i_r50r', 'z_r50r', 'metal_r50r', 'ager_r50r', 

In [101]:
print( LG[1]['subs'].dtype.names )

('nparts', 'id', 'timestep', 'level', 'host', 'hostsub', 'nbsub', 'nextsub', 'aexp', 'm', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'Lx', 'Ly', 'Lz', 'r', 'a', 'b', 'c', 'ek', 'ep', 'et', 'spin', 'sigma', 'rvir', 'mvir', 'tvir', 'cvel', 'rho0', 'rc', 'mcontam', 'r10_mem', 'r50_mem', 'r90_mem', 'r10_vir', 'r50_vir', 'r90_vir', 'r10_max', 'r50_max', 'r90_max', 'sub', 'dink', 'mdm', 'mstar', 'mcold', 'mcell', 'Host')


In [103]:
for key in LG.keys():
    subs = LG[key]['subs']
    real = LG[key]['real']
    for sub in subs:
        where = np.where(real['hid']==sub['id'])[0][0]
        sub['dink'] = real[where]['state']=='dink'
        print("  ",sub['dink'], real[where]['state'])

   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   False upair
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   False pair
   True dink
   False pair
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   False pair
   True dink
   True dink
   False upair
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   False upair
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   True dink
   False pair
   True dink
   True dink
   True dink
   True dink
   False upair
   True dink
   True dink
   False pair
   True dink
   True dink

In [104]:
pklsave(LG, f"{database}/00_LocalGroup_final.pickle", overwrite=True)

In [86]:
newMass

array([(64, 26084, False, 3.95789603e+09, 1.43824562e+08, 2.23168857e+08, 6.19598411e+08)],
      dtype=[('host', '<i4'), ('sub', '<i8'), ('dink', '?'), ('mdm', '<f8'), ('mstar', '<f8'), ('mcold', '<f8'), ('mcell', '<f8')])